In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from generator import RoadNetwork, Trajectory
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
import networkx as nx
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch_geometric.transforms as T

from models import TemporalGraphTrainer, GTCModel, Traj2VecModel, ModelVariant

In [2]:
# 1. Load required data
unmapped_traj = pd.read_csv("../../../datasets/trajectories/hanover/temporal/mapped_id_poly_clipped_small_graph.csv", ";")
traj = Trajectory("../../../datasets/trajectories/hanover/temporal/road_segment_map_final_small_graph.csv", nrows=100000000).generate_TTE_datatset()
traj["seg_seq"] = traj["seg_seq"].map(np.array)
traj = traj.join(unmapped_traj[["start_stamp", "end_stamp", "id"]].set_index("id"), on="id", how="left")
network = RoadNetwork()
network.load_edges("../../../osm_data/hanover_temp_small_graph/")
network.gdf_edges.rename(columns={"speed_limi": "speed_limit", "highway_en": "highway_enc"}, inplace=True)

In [ ]:
# Folium Heat Map
import folium
from folium import plugins
from folium.plugins import HeatMap
from collections import defaultdict
import branca.colormap

m = folium.Map(location=[52.37052, 9.73322],
                    zoom_start = 8)

sub_df["coords"] = sub_df["geometry"].swifter.apply(lambda x: list(x.coords))
coords = sub_df.loc[:, "coords"].values

for line in coords:
    data = [(c[1], c[0]) for c in line]
    folium.PolyLine(data, color="red", weight=2.5, opacity=0.8).add_to(m)

# coords = gdf["coords"].values
# for line in coords:
#     data = [(c[1], c[0]) for c in line]
#     folium.PolyLine(data, color="green", weight=2.5, opacity=0.8).add_to(m)

# Display the map
#map_porto.save("heatmap_gps_points_porto.html")
m

In [3]:
# extract relevant time data from time stamp
traj["start_stamp"] = pd.to_datetime(traj["start_stamp"], unit="s")
traj["end_stamp"] = pd.to_datetime(traj["end_stamp"], unit="s")

traj["dayofweek"] = traj["start_stamp"].dt.dayofweek
traj["start_hour"] = traj["start_stamp"].dt.hour
traj["end_hour"] = traj["end_stamp"].dt.hour

traj

,id,seg_seq,travel_time,start_stamp,end_stamp,dayofweek,start_hour,end_hour
0,1,"[917, 910, 911, 914, 919, 932, 941, 943, 946, ...",655,2019-09-21 13:18:03,2019-09-21 15:21:49,5,13,15
1,19105,"[640, 631, 627, 613, 603, 598, 594, 592, 898, ...",1753,2019-11-23 00:14:00,2019-11-23 02:20:02,5,0,2
2,19107,"[433, 437, 444, 464, 340, 345, 350, 356, 360, ...",313,2019-11-23 00:14:00,2019-11-23 02:20:02,5,0,2
3,4,"[179, 170, 169, 566, 557, 547, 543, 535]",120,2019-11-05 11:01:07,2019-11-05 13:17:07,1,11,13
4,5,"[1010, 1011, 1014]",30,2019-11-05 11:01:07,2019-11-05 13:17:07,1,11,13
...,...,...,...,...,...,...,...,...
22404,19098,"[289, 268, 656]",135,2019-09-28 11:15:56,2019-09-28 11:31:22,5,11,11
22405,19101,"[649, 639, 636, 630, 624, 622, 620, 617, 611]",381,2019-09-28 11:15:56,2019-09-28 11:31:22,5,11,11
22406,19102,"[611, 607, 605, 604, 471, 468, 467, 466, 828]",314,2019-09-28 11:15:56,2019-09-28 11:31:22,5,11,11
22407,19103,"[483, 484, 486, 489, 491, 492, 495, 284, 280, ...",225,2019-09-20 18:17:09,2019-09-20 18:24:54,4,18,18


In [ ]:
# precalc adj matrices
adj = GTCModel.generate_node_traj_adj(network=network, traj_data=traj, k=1, bidirectional=False, add_self_loops=False)
np.savetxt("./traj_adj_k_1_for_temporal_tsd_small_graph.gz", X=adj)

100%|██████████| 22409/22409 [00:06<00:00, 3451.91it/s]


In [5]:
# Pretrain tsd model 
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
adj_t2v = np.loadtxt("./traj_adj_k_1_for_temporal_tsd_small_graph.gz")
print(adj_t2v.shape)
data = network.generate_road_segment_pyg_dataset(only_edge_index=True)
traj2vec = Traj2VecModel(
            data,
            network,
            adj=adj_t2v,
            device=device,
            emb_dim=128,
            walk_length=30,
            context_size=5,
            walks_per_node=25,
            num_neg=10,
        )
traj2vec.train(epochs=30)

(1366, 1366)
Epoch: 1, avg_loss: 13.52697298743508
Epoch: 2, avg_loss: 9.177679755470969
Epoch: 3, avg_loss: 7.557187932910341
Epoch: 4, avg_loss: 6.633694020184603
Epoch: 5, avg_loss: 5.9950261636213815
Epoch: 6, avg_loss: 5.50584077835083
Epoch: 7, avg_loss: 5.108271827945461
Epoch: 8, avg_loss: 4.773991446603428
Epoch: 9, avg_loss: 4.486600589270543
Epoch: 10, avg_loss: 4.235654025728051
Epoch: 11, avg_loss: 4.014046554723061
Epoch: 12, avg_loss: 3.8168586293856297
Epoch: 13, avg_loss: 3.6402153727057924
Epoch: 14, avg_loss: 3.4811373801974503
Epoch: 15, avg_loss: 3.337163337794217
Epoch: 16, avg_loss: 3.2063911191441794
Epoch: 17, avg_loss: 3.087173185883996
Epoch: 18, avg_loss: 2.9781667463707198
Epoch: 19, avg_loss: 2.8781468235134504
Epoch: 20, avg_loss: 2.7861334605650465
Epoch: 21, avg_loss: 2.701243633315676
Epoch: 22, avg_loss: 2.622736357460337
Epoch: 23, avg_loss: 2.549976542768742
Epoch: 24, avg_loss: 2.482372404273712
Epoch: 25, avg_loss: 2.4194265896623786
Epoch: 26, av

In [6]:
traj2vec.save_model(path="")

In [4]:
# load gtc adj
adj = np.loadtxt("./traj_adj_k_2_bi_temporal_gtc_small_graph.gz")

In [ ]:
import gc 
del model
gc.collect()
torch.cuda.empty_cache()

In [5]:
# load tsd pre emb
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
data = network.generate_road_segment_pyg_dataset(only_edge_index=True)
tsd = Traj2VecModel(data, network, device=device)
tsd.load_model("models/model_tsd_small.pt")
tsd_emb = tsd.load_emb()

In [14]:
# adjaceny matrix for tgcn
adj_tgcn = nx.adjacency_matrix(network.line_graph).A

In [6]:
# load train data
data = torch.load("../../../datasets/trajectories/hanover/temporal/temporal_data_small.pt")
data = torch.swapaxes(data, 0, 1)
data = data.numpy()
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
edge_index = network.generate_road_segment_pyg_dataset(only_edge_index=True).edge_index
# initialize model
model = TemporalGraphTrainer(data=data, adj=adj, edge_index=edge_index, struc_emb=tsd_emb, device=device, device_ids=[3], batch_size=16, model_type=ModelVariant.TGTC_ATT, log_name="tgtc-lstm-2layer")

(5952, 1366, 4)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [7]:
model.find_best_lr()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=87` reached.
LR finder stopped early after 87 steps due to diverging loss.
Learning rate set to 0.0005248074602497723
Restoring states from the checkpoint path at /dstore/home/pheinemeyer/Road-Network-Embedding-Generator/models/training/temporal/.lr_find_ffafb770-80af-40ce-82ac-f7f5f7785f60.ckpt


0.0005248074602497723

In [8]:
model.train()

Missing logger folder: tb_logs/tgtc-lstm-2layer
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name     | Type                     | Params
------------------------------------------------------
0 | encoder  | TemporalAttentionEncoder | 266 K 
1 | decoder  | TemporalGraphDecoder     | 140 K 
2 | loss_seq | MSELoss                  | 0     
------------------------------------------------------
406 K     Trainable params
0         Non-trainable params
406 K     Total params
1.627     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Metric val_loss improved. New best score: 2.057


Validation: 0it [00:00, ?it/s]

Metric val_loss improved by 0.159 >= min_delta = 0.005. New best score: 1.898


In [ ]:
%load_ext tensorboard
%tensorboard --logdir tb_logs


In [11]:
model.load_model("model_small_current_best_normal_gnorm.pt")

FileNotFoundError: [Errno 2] No such file or directory: 'model_small_current_best_normal_gnorm.pt'

In [15]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

count_parameters(model.model)

+--------------------------------------+------------+
|               Modules                | Parameters |
+--------------------------------------+------------+
| encoder._encoder.graph_conv1.weights |   50688    |
| encoder._encoder.graph_conv1.biases  |    384     |
| encoder._encoder.graph_conv2.weights |   16896    |
| encoder._encoder.graph_conv2.biases  |    128     |
|    decoder._tdecoder.weight_ih_l0    |   65536    |
|    decoder._tdecoder.weight_hh_l0    |   65536    |
|     decoder._tdecoder.bias_ih_l0     |    512     |
|     decoder._tdecoder.bias_hh_l0     |    512     |
|        decoder.dense.0.weight        |    8192    |
|         decoder.dense.0.bias         |     64     |
|        decoder.dense.2.weight        |     64     |
|         decoder.dense.2.bias         |     1      |
+--------------------------------------+------------+
Total Trainable Params: 208513


208513

In [ ]:
# train model
model.train(epochs=10)

In [ ]:
torch.save(model.model.state_dict(), os.path.join("tgtc-lstm-2layer.pt"))

In [21]:
device

device(type='cuda', index=2)

In [24]:
# test sequence encoding 
model.model.to(device).eval()
z, _ = model.model(torch.Tensor(data[100:112]).unsqueeze(0).to(device))
z = z.squeeze()
z.shape

torch.Size([1366, 128])

In [25]:
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

# n2v = models[-1]
idxs = np.arange(len(network.line_graph.nodes))
train_idx, test_idx = model_selection.train_test_split(idxs, test_size=0.2, random_state=69)
y = np.array([network.gdf_edges.loc[n]["highway_enc"] for n in network.line_graph.nodes])

# for m, e in models:
    # m.train(epochs=e)
    # zn = m.load_emb()
    # zcn = np.concatenate((zn, z2), axis=1)
    # zct = np.concatenate((zn, z3), axis=1)
    # zcnn = np.concatenate((zn, z4), axis=1)
    # zctn = np.concatenate((zn, z5), axis=1)
    # X = z # embedding for each node
eva = [z.detach().cpu().numpy()] # gtc.load_emb(), gae_emb, rand_emb
for X in eva:
    X_train, X_test, y_train, y_test = X[train_idx], X[test_idx], y[train_idx], y[test_idx]

    lm = linear_model.LogisticRegression(multi_class="multinomial", max_iter=1000)
    scorer = make_scorer(metrics.f1_score, average="macro")
    print(np.mean(cross_val_score(estimator=lm, X=X, y=y, scoring=scorer, cv=5)))

0.6549915452310643


# Test on traveltime task

In [32]:
from torch.utils.data import DataLoader, Dataset

from sklearn import model_selection


class TemporalDataset(Dataset):
    def __init__(self, data, network):
        self.X = data["seg_seq"].values
        self.y = data["travel_time"].values
        self.time = data[["dayofweek", "start_hour", "end_hour"]].values
        self.network = network
        self.map = self.create_edge_emb_mapping()

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return torch.tensor(self.X[idx], dtype=int), self.y[idx], self.time[idx], self.map

    # tested index mapping is correct
    def create_edge_emb_mapping(self):
        # map from trajectory edge id to embedding id
        # edge_ids = np.array(self.network.gdf_edges.index, dtype="i,i,i")
        # traj_edge_idx = np.array(self.network.gdf_edges.fid)
        # node_ids = np.array(self.network.line_graph.nodes, dtype="i,i,i")
        # sort_idx = node_ids.argsort()
        # emb_ids = sort_idx[np.searchsorted(node_ids, edge_ids, sorter=sort_idx)]
        # map = dict(zip(traj_edge_idx, emb_ids))

        map = {}
        nodes = list(self.network.line_graph.nodes)
        for index, id in zip(self.network.gdf_edges.index, self.network.gdf_edges.fid):
            map[id] = nodes.index(index)
        # print(map == map2) # yields true

        return map

    @staticmethod
    def collate_fn_padd(batch):
        """
        Padds batch of variable length
        """
        data, label, time, map = zip(*batch)
        # seq length for each input in batch
        lengths_old = torch.tensor([t.shape[0] for t in data])

        # sort data for pad packet, since biggest sequence should be first and then descending order
        sort_idxs = torch.argsort(lengths_old, descending=True, dim=0)
        lengths = lengths_old[sort_idxs]
        data = [
            x
            for _, x in sorted(
                zip(lengths_old.tolist(), data), key=lambda pair: pair[0], reverse=True
            )
        ]
        label = [
            x
            for _, x in sorted(
                zip(lengths_old.tolist(), label), key=lambda pair: pair[0], reverse=True
            )
        ]
        time = [
            x
            for _, x in sorted(
                zip(lengths_old.tolist(), time), key=lambda pair: pair[0], reverse=True
            )
        ]

        # pad
        data = torch.nn.utils.rnn.pad_sequence(data, padding_value=0, batch_first=True)
        # compute mask
        mask = data != 0

        return data, torch.Tensor(label), torch.tensor(time, dtype=int), lengths, mask, map[0]


train, test = model_selection.train_test_split(
            traj, test_size=0.2, random_state=69
        )
train_loader = DataLoader(
    TemporalDataset(train, network),
    collate_fn=TemporalDataset.collate_fn_padd,
    batch_size=32,
    shuffle=True,
)
eval_loader = DataLoader(
    TemporalDataset(test, network),
    collate_fn=TemporalDataset.collate_fn_padd,
    batch_size=32,
)

In [33]:
from operator import itemgetter
from torch.autograd import Variable

# Init 
class TemporalTest(nn.Module):
    def __init__(
        self,
        device,
        emb_dim: int = 128,
        hidden_units: int = 128,
        layers: int = 2,
        batch_size: int = 128,
        plugin=None,
    ):
        super(TemporalTest, self).__init__()
        self.encoder = nn.LSTM(
            emb_dim, hidden_units, num_layers=layers, batch_first=True, dropout=0.5
        )
    
        self.decoder = nn.Sequential(
            nn.Linear(hidden_units, hidden_units * 2),
            nn.ReLU(),
            nn.Linear(hidden_units * 2, hidden_units),
            nn.ReLU(),
            nn.Linear(hidden_units, 1),
        )
        self.hidden_units = hidden_units
        self.layers = layers
        self.batch_size = batch_size
        self.device = device
        self.loss = nn.MSELoss()
        self.opt = torch.optim.Adam(self.parameters(), lr=0.0001)
        self.plugin = plugin

        self.encoder.to(device)
        self.decoder.to(device)

    def forward(self, x, lengths):
        batch_size, seq_len, _ = x.size()
        self.hidden = self.init_hidden(batch_size=batch_size)

        x = torch.nn.utils.rnn.pack_padded_sequence(x, lengths, batch_first=True)

        x, _ = self.encoder(x)

        x, plengths = torch.nn.utils.rnn.pad_packed_sequence(
            x, batch_first=True, padding_value=0
        )
        x = x.contiguous()  # batch x seq x hidden
        # x = x.view(-1, x.shape[2])
        x = torch.stack(
            [x[b, plengths[b] - 1] for b in range(batch_size)]
        )  # get last valid item per batch batch x hidden


        yh = self.decoder(x)

        return yh  # (batch x 1)

    def train_model(self, loader, epochs=100):
        self.train()
        for e in range(epochs):
            total_loss = 0
            for X, y, time, lengths, mask, map in loader:

                emb, _ = self.plugin.generate_emb(time)
    
                emb_batch = self.get_embedding(emb.detach().cpu(), X.clone(), mask, map)
                emb_batch = emb_batch.to(self.device)

                y = y.to(self.device)
                yh = self.forward(emb_batch, lengths)
                loss = self.loss(yh.squeeze(), y)

                self.opt.zero_grad()
                loss.backward()
                self.opt.step()
                total_loss += loss.item()


            print(f"Average training loss in episode {e}: {total_loss/len(loader)}")

    def predict(self, loader):
        with torch.no_grad():
            self.eval()
            yhs, ys = [], []
            for X, y, time, lengths, mask, map in loader:
                emb, _ = self.plugin.generate_emb(time)

                emb_batch = self.get_embedding(emb.detach().cpu(), X.clone(), mask, map)
                emb_batch = emb_batch.to(self.device)

                y = y.to(self.device)
                yh = self.forward(emb_batch, lengths)
                yhs.extend(yh.tolist())
                ys.extend(y.tolist())

            return np.array(yhs), np.array(ys)

    def init_hidden(self, batch_size):
        hidden_a = torch.randn(self.layers, batch_size, self.hidden_units)
        hidden_b = torch.randn(self.layers, batch_size, self.hidden_units)

        hidden_a = Variable(hidden_a)
        hidden_b = Variable(hidden_b)

        hidden_a = hidden_a.to(self.device)
        hidden_b = hidden_b.to(self.device)

        return (hidden_a, hidden_b)

    def get_embedding(self, emb, batch, mask, map):
        """
        Transform batch_size, seq_length, 1 to batch_size, seq_length, emb_size
        """
        res = torch.zeros((batch.shape[0], batch.shape[1], emb.shape[-1]))
        for i, seq in enumerate(batch):
            idx = i if emb.shape[0] > 1 else 0
            emb_ids = itemgetter(*seq[mask[i]].tolist())(map)
            res[i, mask[i], :] = emb[idx, emb_ids, :]

        return res


In [ ]:
# load temporal sensor data 
temporal = pd.read_csv("../../../datasets/trajectories/hanover/temporal/hannover_traffic.csv")

In [35]:
# initialize temporal plugin 
from evaluation.tasks import TemporalEmbeddingPlugin

plugin = TemporalEmbeddingPlugin(model.model, network, device)

In [36]:
plugin.load_data("plugin_data_small.pt")

In [18]:
plugin.processed_temp_data.shape

torch.Size([768, 1366, 4])

In [62]:
# save preprocessed 
torch.save(plugin.processed_temp_data, 'plugin_data_small.pt')

In [37]:
for i, t in tqdm(traj.iterrows()):
    z, _ = plugin.generate_emb(t.values[-3:][np.newaxis, ...])

22409it [02:43, 136.77it/s]


In [19]:
print(z.shape)

torch.Size([1, 1366, 128])


In [38]:
task = TemporalTest(
            plugin=plugin,
            device=device,
            batch_size=32,
            emb_dim=128,
            hidden_units=128
        )

In [39]:
task.train_model(loader=train_loader, epochs=10)

Average training loss in episode 0: 87829.1031143814
Average training loss in episode 1: 22002.679073884527
Average training loss in episode 2: 19545.130449862827
Average training loss in episode 3: 18723.289141704267
Average training loss in episode 4: 18429.44821215951
Average training loss in episode 5: 18138.19066773549
Average training loss in episode 6: 18028.32034339837
Average training loss in episode 7: 17895.940817440256
Average training loss in episode 8: 17518.16926865739
Average training loss in episode 9: 17453.806573606005


In [40]:
yh, y = task.predict(loader=eval_loader)
print(metrics.mean_absolute_error(yh, y), metrics.mean_squared_error(yh, y, squared=False))

79.77319039153713 129.8692814147533


lstm -> 80.862
lstm_tsd -> 78.73

85.69705600006567 143.56656361147427


# Test on gcn/node2vec

In [4]:
from models import GAEModel, GCNEncoder, GATEncoder, Node2VecModel

In [10]:
data = network.generate_road_segment_pyg_dataset(traj_data=None, include_coords=False, dataset="hannover_small")

In [11]:
from torch_geometric.data import Data
import torch_geometric.transforms as T

# create pyg dataset
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
transform = T.Compose([
    # T.OneHotDegree(128), # training without features
    T.ToDevice(device),
])
data = transform(data)
gae = GAEModel(data, device=device, encoder=GATEncoder, emb_dim=128)
gae.train(epochs=5000)

Epoch: 500, avg_loss: 1.1257418394088745
Epoch: 1000, avg_loss: 1.0708516818881035
Epoch: 1500, avg_loss: 1.0423320540189742
Epoch: 2000, avg_loss: 1.0241585468947887
Epoch: 2500, avg_loss: 1.0099976078748703
Epoch: 3000, avg_loss: 0.9990644570589066
Epoch: 3500, avg_loss: 0.990465463416917
Epoch: 4000, avg_loss: 0.9834868279695511
Epoch: 4500, avg_loss: 0.9776899226374096


In [12]:
gae.save_model(".")

In [13]:
gae_z = gae.model.encode(data.x, data.edge_index).detach().cpu().numpy()

In [25]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
transform = T.Compose([
    T.ToDevice(device),
])
data = transform(data)
n2v = Node2VecModel(data, device=device, q=1, p=1, negative_samples=3)
n2v.train(epochs=20)

In [26]:
n2v.save_model("")

In [15]:
n2v_z = n2v.load_emb()

In [27]:
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

# n2v = models[-1]
idxs = np.arange(len(network.line_graph.nodes))
train_idx, test_idx = model_selection.train_test_split(idxs, test_size=0.2, random_state=69)
y = np.array([network.gdf_edges.loc[n]["highway_enc"] for n in network.line_graph.nodes])

# for m, e in models:
    # m.train(epochs=e)
    # zn = m.load_emb()
    # zcn = np.concatenate((zn, z2), axis=1)
    # zct = np.concatenate((zn, z3), axis=1)
    # zcnn = np.concatenate((zn, z4), axis=1)
    # zctn = np.concatenate((zn, z5), axis=1)
    # X = z # embedding for each node
eva = [gae_z, n2v_z, np.concatenate([gae_z, n2v_z], axis=-1)] # gtc.load_emb(), gae_emb, rand_emb
for X in eva:
    X_train, X_test, y_train, y_test = X[train_idx], X[test_idx], y[train_idx], y[test_idx]

    lm = linear_model.LogisticRegression(multi_class="multinomial", max_iter=1000)
    scorer = make_scorer(metrics.f1_score, average="macro")
    print(np.mean(cross_val_score(estimator=lm, X=X, y=y, scoring=scorer, cv=5)))

0.4663178397023019
0.21722996450955182
0.5198928335690597


In [30]:
class NormalPlugin:
    def __init__(self, model: nn.Module, network, device):
        super().__init__()
        self.model = model
        self.device = device
        self.network = network

    def generate_emb(self, times):
        # Bx3 - 3 time features B -> Batch Size
        return torch.Tensor(np.concatenate([self.model[0].load_emb(), self.model[1].load_emb()], axis=-1)).unsqueeze(0), None

plugin = NormalPlugin((gae, n2v), network, device)

In [31]:
emb, _ = plugin.generate_emb(_)

emb.shape

torch.Size([1, 1366, 256])

In [43]:
task = TemporalTest(
            plugin=plugin,
            device=device,
            batch_size=64,
            hidden_units=256,
            emb_dim=256
        )

In [44]:
task.train_model(loader=train_loader, epochs=20)

Average training loss in episode 0: 58252.96916560481
Average training loss in episode 1: 20444.590375550077
Average training loss in episode 2: 19621.356012300163
Average training loss in episode 3: 19542.15976719168
Average training loss in episode 4: 19454.929917279413
Average training loss in episode 5: 19105.126787753452
Average training loss in episode 6: 18854.38008256392
Average training loss in episode 7: 18727.521045705213
Average training loss in episode 8: 18596.11075933392
Average training loss in episode 9: 18494.71521844711
Average training loss in episode 10: 18593.629486791164
Average training loss in episode 11: 18346.102795823586
Average training loss in episode 12: 18313.28846063461
Average training loss in episode 13: 18075.04020835074
Average training loss in episode 14: 18244.0101777483
Average training loss in episode 15: 17858.406213879374
Average training loss in episode 16: 17779.946660713293
Average training loss in episode 17: 17683.174168442234
Average tra

In [46]:
yh, y = task.predict(loader=eval_loader)
print(metrics.mean_absolute_error(yh, y), metrics.mean_squared_error(yh, y, squared=False))

84.79005843174878 134.66384316477473
